### 1、获取电影首页页面海报
```
字段如下：
movie_id：影片ID
movie_url：影片连接
movie_home_poster：首页影片海报
```

In [ ]:
# 用fake_useragent 库中的 UserAgent 类来生成一个虚假的用户代理，以模拟浏览器的行为
from fake_useragent import UserAgent
# 用了 urllib.parse 模块中的 quote 函数来对URL进行编码
# lxml 模块用于解析HTML响应
import requests
import time
# 请求数据函数
def request_data(url, headers, params):
    res = None
    num = 0
    # 设置请求失败次数，不超过5次
    while num <= 5:
        response = requests.get(url, headers=headers, params=params)
        response.encoding = "utf-8"
        if response.status_code == 200:
            res = response.json()
            break
        else:
            num += 1
            continue
    return res


# 保存数据
data_save = open(file="../data/home_page_data",mode="a",encoding="utf-8")
# 定义列名
name_list = 'movie_id$movie_url$movie_home_poster'+'\n'
data_save.write(name_list)
def get_data():
    ua = UserAgent().random
    headers = {
        'User-Agent': ua,
        'Referer': 'https://movie.douban.com/explore'
    }
    url_base = 'https://m.douban.com/rexxar/api/v2/movie/recommend?'
    for year in ['2019','2020','2021','2022','2023']:
        params = {
            'refresh': '0',
            'start': '0',
            'count': '350',
            'selected_categories': '{}',
            'uncollect': 'false',
            'tags': year
        }
        data = request_data(url_base, headers, params)
        for index in data['items']:
            # 电影id
            movie_id = index['id']
            # 电影url
            movie_url = 'https://movie.douban.com/subject/'+ movie_id + '/'
            # 海报url
            poster_url = requests.get(url=index['pic']['large']).content
            # 电影海报保存路径
            file_path =  '../data/image/' + movie_id + '/'
            # 以这路径创建文件夹
            if not os.path.exists(file_path):
                os.makedirs(file_path)
            # 电影海报图名称名称
            movie_home_poster = file_path + 'p' + movie_id
            with open(movie_home_poster + '.jpg', mode='wb') as f2:
                f2.write(poster_url)
            res = movie_id + '$' +movie_url + '$' + movie_home_poster + '\n'
            data_save.write(res)
            print(res)
            # 间隔1秒请求1次
            time.sleep(1)

if __name__ == '__main__':
    data = get_data()

### 2、获取电影详细页面数据
```
字段如下：
movie_id：影片ID
movie_name：影片名称
movie_director：影片导演
movie_writer：影片编剧
movie_starring：影片主演
movie_type：影片类型
movie_country：影片制片国家
movie_language：影片语言
movie_release_date：影片上映日期
movie_release_place 影片上映地方
movie_run_time：影片片长
movie_rating：影片评分
rating_people_num：评分人数
film_synopsis：影片剧情简介
```

In [ ]:
# 解释html和定位元素
from bs4 import BeautifulSoup
import pandas as pd

# 保存数据
data_save = open(file="../data/detil_page_data.csv",mode="a+",encoding="utf-8")
# 定义列名
name_list = 'movie_id$movie_name$movie_director$movie_writer$movie_starring$movie_type$\
movie_country$movie_language$movie_release_date$movie_release_place$movie_run_time$movie_rating$\
rating_people_num$film_synopsis' + '\n'
data_save.write(name_list)

# 批量读取url
file_path = pd.read_csv("../data/home_page_data",sep="$")
for url in file_path['movie_url']:
    ua = UserAgent().random
    headers = {
        'User-Agent': ua,
        'Cookie': 'll="118281"; bid=EIN7y1TWjCk; ct=y; push_noty_num=0; push_doumail_num=0; douban-fav-remind=1; ap_v=0,6.0'
    }
    response = requests.get(url, headers)
    response.encoding = "utf-8"
    if response.status_code == 200:
        # 打印整个HTML文档的树状结构
        soup = BeautifulSoup(response.text, 'html.parser')
        # 提取影片ID
        movie_id = soup.find("div",id="mainpic").find("a",class_="nbgnbg")["href"].split("subject")[1].split("photos")[0].strip("/")
        # 提取影片名称
        movie_name = soup.find("div",id="content").find("h1").find("span").text
        # 提取影片导演
        movie_director = soup.find(id='info').find('span').find("span",class_="attrs").find("a").text.strip()
        # 提取影片编剧
        movie_writer = soup.find(id='info').find_all('span')[3].text.replace("\n","").replace(" ","").strip()
        # 提取影片主演
        movie_starring = soup.find(id='info').find_all('span')[6].text.replace("\n","").replace(" ","")[:-5]+'...'
        # 提取影片类型
        movie_type = "/".join([i.text for i in soup.find(id='info').find_all('span', property='v:genre')])
        # 提取影片制片国家
        movie_country = soup.find(id='info').find(text='制片国家/地区:').next_element.strip().replace("\n","").replace(" ","")
        # 提取影片语言
        movie_language = soup.find(id='info').find(text='语言:').next_element.strip().replace("\n","").replace(" ","")
        # 提取影片上映数据
        release_data = soup.find(id='info').find_all('span', property='v:initialReleaseDate')[0].text
        # 上映日期
        movie_release_date = release_data.strip(")").split("(")[0]
        # 上映地方
        movie_release_place = release_data.strip(")").split("(")[1]
        # 影片片长
        movie_run_time = soup.find(id='info').find_all('span', property='v:runtime')[0].text
        # 影片评分
        movie_rating = soup.find("div",class_="rating_wrap clearbox").find("div",class_="rating_self clearfix")\
            .find("strong").text.replace("\n","").strip()
        # 评分人数
        rating_people_num = soup.find("div",class_="rating_wrap clearbox").find("div",class_="rating_self clearfix")\
            .find("div",class_="rating_right").find("div",class_="rating_sum").find("span").text.replace("\n","").strip()

        # 提取电影剧情简介
        film_synopsis = soup.find("div",class_="related-info").find("div",class_="indent")\
            .find("span").text.replace("\n","").replace(" ","").strip()
        # 合并结果
        res = movie_id + '$' + movie_name + '$' + movie_director + '$' + movie_writer + '$' + movie_starring + '$' + \
            movie_type + '$' + movie_country + '$' + movie_language + '$' + movie_release_date + '$' + movie_release_place + '$' + \
            movie_run_time + '$' + movie_rating + '$' + rating_people_num + '$' + film_synopsis + '\n'
        # 将结果输出
        data_save.write(res)
        print(res)
    else:
        print("请求失败，状态码：", response.status_code)
        # 请求数据失败时数值赋值为"None"
        res = 'None$None$None$None$None$None$None$None$None$None$None$None$None$None$None$None$None' + '\n'
        # 将结果输出
        data_save.write(res)

#### 3、电影评论数据
```
字段如下：
movie_id：影片ID
user_id：用户ID
user_rating：用户评分
timestamp：评论时间
```

In [ ]:
import requests  
from lxml import etree
import pandas as pd

# 保存数据
data_save = open(file="../data/comment_page_data.csv",mode="a+",encoding="utf-8")
# 定义列名
name_list = 'movie_id$user_id$user_rating$time_stamp'+'\n'
data_save.write(name_list)

# 电影评论url
file_path = pd.read_csv("../data/home_page_data",sep="$")
for url in file_path['movie_url']:
    ua = UserAgent().random
    headers = {
        'User-Agent': ua,
        'Cookie': ''
    }
    for score in ['h','m','l']:
        params = {
            'percent_type': score,
            'limit': '600',
            'status': 'P',
            'sort': 'new_score',
        }
        response = requests.get(url=url, params=params, headers=headers)
        response.encoding = "utf-8"
        if response.status_code == 200:
            # 使用lxml解析响应内容  
            tree = etree.HTML(response.content)  
            # movie_id：影片ID
            movie_id = [mid.replace("\n","").strip().split("subject")[1].split("comments?")[0]\
            .strip("/") for mid in tree.xpath('//*[@id="comments"]/div/div[2]/div/@data-url')]
            # user_id：用户ID
            user_id = [uid.replace("\n","").strip().split("people")[1].strip("/") for uid in tree.xpath('//*[@id="comments"]/div/div[1]/a/@href')]
            # user_rating：评分
            comm_rating = [comm_rat.replace("\n","").strip() for comm_rat in tree.xpath('//*[@id="comments"]/div/div[2]/h3/span[2]/span[2]/@class')]
            # 评论时间戳(将标准时间格式转换为时间戳)
            time_stamp = [str(int(time.mktime(time.strptime(time_str.strip(), "%Y-%m-%d %H:%M:%S")))) for time_str in tree.xpath('//*[@id="comments"]/div/div[2]/h3/span[2]/span[3]/@title')]
            # 取这4个字段长度最小作为索引值(目的解决list index out of range)
            list_num = [len(movie_id),len(user_id),len(comm_rating),len(time_stamp)]
            list_num.sort()
            for index_num in range(list_num[0]):
                res = movie_id[index_num] + '$' + user_id[index_num] + '$' + comm_rating[index_num] + '$' + time_stamp[index_num] + '\n'
                data_save.write(res)
                print(res)
        else:
            print("请求失败，状态码：", response.status_code)
            res = 'None$None$None$None' + '\n'
            data_save.write(res)
            print(res)

#### 4、用户标签数据集
```
字段如下：
movie_id：影片ID
user_id：用户ID
user_tag：用户标签
timestamp：评论时间
```

In [12]:
import pandas as pd

# 将电影数据和用户评论数据，以movie_id为主键合并数据集
data1 = pd.read_csv("../data/detil_data.csv",sep="$")
data2 = pd.read_csv("../data/comment_data.csv",sep="$")
sub_data = data1.merge(data2,on="movie_id",how="inner")
df = sub_data[['movie_id','user_id','movie_type','time_stamp']].drop_duplicates()
# 将 movie_type 列中的每个类型以 '/' 分割，并将结果保存到新的行中  
new_rows = []
for index, row in df.iterrows():
    try:
        types = row['movie_type'].split('/')
        for type in types:
            new_rows.append([row['movie_id'], row['user_id'], type, row['time_stamp']])  
    except Exception as e:
        for type in types:
            new_rows.append([row['movie_id'], row['user_id'], "None", row['time_stamp']])  
# 创建新的 DataFrame，并将新的行添加到其中，并movie_type列改为user_tag
new_df = pd.DataFrame(new_rows, columns=df.columns).rename(columns={"movie_type":"user_tag"})
new_df.to_csv("../data/tag_data.csv",sep="$",index=False)

In [ ]:
len(new_df)

In [3]:
recall = 5.6329497534853905
precision = 0.8841807909604562
F1 = (2*precision*recall)/(precision+recall)
F1

1.5284475075374502

In [6]:
import pandas as pd
data = pd.read_csv("../data/wp/Tag-1.csv",sep=",",index_col=False)
print(data['user_tag'].to_list())

['喜剧', '悬疑', '犯罪', '惊悚', '恐怖', '剧情', '科幻', '动画', '冒险', '历史', '动作', '奇幻', '武侠', '古装', '爱情', '纪录片', '运动', '同性', '战争', '音乐', '灾难', '传记', '歌舞', '家庭', '短片', '儿童', '真人秀', '戏曲', '情色', '脱口秀', '西部']


In [7]:
a = ['喜剧', '悬疑', '犯罪', '惊悚', '恐怖', '剧情', '科幻', '动画', '冒险', '历史', '动作', '奇幻', '武侠', '古装', '爱情', '纪录片', '运动', '同性', '战争', '音乐', '灾难', '传记', '歌舞', '家庭', '短片', '儿童', '真人秀', '戏曲', '情色', '脱口秀', '西部']
len(a)

31

In [ ]:
a = 10